# PROJETO FINAL DE PROCESSAMENTO DIGITAL DE SINAIS II   <br>
**CURSO DE ENGENHARIA ELETRÔNICA**                      <br>
**INSTITUTO FEDERAL DE SANTA CATARINA**                 <br>
**PROFESSOR: FERNANDO PACHECO**                         <br>
**ALUNO: ERIC MONTEIRO DOS REIS**                       <br><br>

In [1]:
import os
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
import glob
from pdf2image import convert_from_path
from PIL import Image


In [2]:
#FUNCTION TO HELP VISUALIZE AND DEBUG THE PROJECT, TO ADJUST DESIRED SIZE, MODIFY FIGSIZE
def display_img(img):
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')

In [3]:
#DETECTS THE LINES ON THE IMAGE USING THE PROBABILISTIC HOUGH TRANSFORM
def hough_lines(img):


    dst = cv2.Canny(img, 50, 200, None, 3)  #Edge Detection on the image   
    #display_img(dst) #test purposes
    linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 300, 100)
    return linesP

In [4]:
#CALCULATES THE ANGLE OF ROTATION 

def calculate_angle(lines):
    angles = [] #creating a list to save the angles
    for lines in lines:
        for x1,y1,x2,y2 in lines:
            angle_rad = math.atan2(y2-y1, x2 - x1) #finds the angle in radians on the current line
            angle_deg = math.degrees(angle_rad) #converts to degrees
            angles.append(angle_deg) #adds the current angle to the list of angles created
    
    angles = [angle for angle in angles if np.abs(angle) < 10] 
    median_angle = np.median(angles) 

    if (45<median_angle<90):
        final_angle = 90 - median_angle
    elif (-90<median_angle<-45):
        final_angle = -90 - median_angle
    else:
        final_angle = - (median_angle)

    print("Tilt angle: ",final_angle)
    return final_angle

Para eliminarmos os efeitos da rotação na imagem precisamos aplicar um *frame* para corrigirmos a imagem e as partes em preto não causarem a sensação de a imagem estar rotacionada, mesmo ela possuindo uma correção no ângulo. Na imagem abaixo demonstramos visualmente o processo aplicado para obtermos a máscara que irá ser aplicada na imagem no eixo y (altura).

![<caption>](explicacao_do_frame.png)

A imagem deslocada possui um aumento de pixels no eixo da altura e comprimento (2° passo), porém os espaços fora do conteúdo rotacionado ficam pretos. Para eliminarmos isso inicialmente fazemos um *crop* na imagem, utilizando a metade da diferença de tamanho das dimensões da imagem rotacionada subtraídas do tamanho da imagem original (3°passo). Utilizando trigonometria podemos calcular o valor desejado de pixels para aplicarmos a máscara e eliminarmos o preto desejado dentro da página rotacionada.

O processo completo descrito acima é aplicado na função *frame_correction()* 

In [5]:
#THIS FUNCTION USES THE ANGLE THAT THE IMAGE WAS ROTATED AND CHOSES A FRAME TO TAKE AWAY THE EFFECTS OF THE ROTATION ON THE IMAGE
def frame_correction(img_pre,img,angle): 
    if(angle < 0):
        angle = -angle        #the sin function used later needs a positive angle to work properly  
    angle = np.radians(angle) 
    #print(img_pre.shape) #
    #print(img.shape)
    orig_height, orig_width = img_pre.shape[:2] 
    rot_height, rot_width = img.shape[:2] 
    
    half_delta_height = int((rot_height - orig_height)/2)
    half_delta_width = int((rot_width - orig_width)/2)

    mask_height = int(np.sin(angle)*orig_width - half_delta_height)
    mask_width = int(np.sin(angle)*orig_height - half_delta_width)
    

#BUILDING THE FRAME TO ELIMINATE THE ROTATION EFFECT
#cropped = img[start_row:end_row, start_col:end_col]
    new_img = img[half_delta_height:(orig_height+half_delta_height),half_delta_width:(orig_width+half_delta_width)]
    
    new_img[0:mask_height,0:orig_width] = 255
    new_img[orig_height-mask_height:orig_height,0:orig_width] = 255
    new_img[0:orig_height,0:mask_width] = 255
    new_img[0:orig_height,orig_width-mask_width:orig_width] = 255
    return(new_img)

In [6]:
#CREATING FOLDERS FOR THE PROCESSES IMAGES AND THE PDF OUTPUT
path = input("Please insert the folder where the pdf is located") #OBS: IF THERES A SPECIAL CHARACTER IN THE PATH INPUTED, IMPORTING THE IMAGES LATER WILL NOT BE POSSIBLE
new_folder1 = "pdf_output"
new_folder2 = "pdf_images"
os.chdir(path)

try:
    if not os.path.exists(new_folder1 and new_folder2):
        os.makedirs(new_folder1)
        os.makedirs(new_folder2)
except OSError:
    print("Folder already exists")

# INSERT BELOW INSIDE THE FUNCTION THE NAME OF THE PDF WITH THE EXTENSION ALSO, EXAMPLE: "N3_bunpou.pdf"
book_pages = convert_from_path('N3_bunpou.pdf',500)

A função de importar o pdf da biblioteca "pdf2image" só funcionou inserindo diretamente o nome na função, utilizando variáveis ela apresentava erro, portanto ficou definido que o programa irá criar uma pasta de output para salvar o pdf e também uma pasta para salvar as imagens tratadas no local onde o pdf se encontra. Importante observar também que a função não funciona bem lendo PDF's muito pesados, podendo inclusive não concluir o carregamento do pdf porque a função acaba consumindo memória demais do computador.

In [7]:
#converting the pdf to images and enumerating the pages on the format "page ..."
os.chdir(path + "/" + new_folder2)
for i in range(len(book_pages)):
    book_pages[i].save('page' + str(i).zfill(2) + '.png') #using str.zfill() so our numbers start 00 instead of 0, to make sorting out the list of images easier later on the code 

In [8]:
#READING THE PAGES ON THE PATH SPECIFIED WHERE THEY ARE SAVED AND SAVING ON A LIST
#OBS: IF THERES A SPECIAL CHARACTER IN THE PATH INPUTED THE GLOB FUNCTION DOESN'T WORK AS INTENDED

page_search = sorted(glob.glob("*.png")) #searches for the png files
pages_list = []

for file in page_search:
    img = cv2.imread(file)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    pages_list.append(img)

In [9]:
#LOOP RESPONSIBLE FOR APPLYING THE HOUGH PROBABILISTIC TRANSFORM AND CALCULATING THE ROTATION AND SAVING IT ON THE OUTPUT LIST

output_list = []
for i,page in enumerate(pages_list):
    linesP = hough_lines(page)
    tilt_angle = calculate_angle(linesP)
    final = imutils.rotate_bound(page,tilt_angle)
    #display_img(final) #debugging purposes
    result = frame_correction(page,final,tilt_angle)
    thresh = cv2.threshold(result, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    #cv2.imwrite(f"p{i}.png",thresh) #debugging purposes
    output_list.append(thresh)

Tilt angle:  -0.0
Tilt angle:  -0.0
Tilt angle:  -1.0056495363047109
Tilt angle:  0.9970146614618153
Tilt angle:  -0.9992436702572906
Tilt angle:  1.9825238610843805
Tilt angle:  -1.9810703917474939
Tilt angle:  1.9866782381783066
Tilt angle:  -1.980320808902372
Tilt angle:  1.9978798564766747
Tilt angle:  -1.9820760927623546
Tilt angle:  1.8755140525124108
Tilt angle:  -0.9999274764562817
Tilt angle:  1.0751297105970008
Tilt angle:  -1.9690765429684238
Tilt angle:  1.039396156451588
Tilt angle:  -1.8595993387455583
Tilt angle:  0.9554540036369561
Tilt angle:  -1.0038716485184662
Tilt angle:  0.9977936825860246


In [10]:
#INSERTING THE PROCESSED IMAGES BACK IN A PDF ON THE OUTPUT FOLDER

os.chdir(path + "/" + new_folder1) 
pdf_path = os.getcwd() + "/" + "Result.pdf"

#necessary to convert the images to the Pillow library image format so we are able to export the pdf
for i in range(len(output_list)):
    output_list[i] = Image.fromarray(output_list[i])
    
output_list[0].save(pdf_path, resolution=400.0,save_all=True, append_images=output_list[1:])